In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
import cPickle as pickle
import numpy as np
import tensorflow as tf
#from sklearn import metrics
import pickle
from lib import f1_score

In [2]:
data_dir = "../data/"

with open(data_dir + 'tf_data_mean.pickle', 'rb') as f:
    save = pickle.load(f)
with open(data_dir + 'tf_data_max.pickle', 'rb') as f:
    save_max = pickle.load(f)   
    

train_dataset = np.matrix(save['train_data'], dtype='float32')
train_labels = np.matrix(save['train_labels'], dtype='float32')
valid_dataset = np.matrix(save['validate_data'], dtype='float32')
valid_labels = np.matrix(save['validate_labels'], dtype='float32')
test_dataset = np.matrix(save['test_data'], dtype='float32')
test_bids = list(save['test_business_ids'])
#test_bids = np.ravel(test_bids)
del save  # hint to help gc free up memory

print "merging valid max and mean"
valid_dataset = np.concatenate((valid_dataset, save_max['validate_data']), axis=1)

print "merging valid max and mean"
train_dataset = np.concatenate((train_dataset, save_max['train_data']), axis=1)

print "merging valid max and mean"
test_dataset = np.concatenate((test_dataset, save_max['test_data']), axis=1)



print 'Training set', train_dataset.shape, train_labels.shape
print 'Validation set', valid_dataset.shape, valid_labels.shape
print 'Test set', test_dataset.shape, len(test_bids)

merging valid max and mean
merging valid max and mean
merging valid max and mean
Training set (1600, 2048) (1600, 9)
Validation set (400, 2048) (400, 9)
Test set (10000, 2048) 10000


In [33]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.

#set to -1 to use all data
train_subset = -1
input_size = 2048
labels_size = 9
layer_1_units = 512
layer_2_units = 512
dropout_keep_prob = 1.

#def f1_score(y_true, y_pred):
#    l, w = y_pred.shape
#    y_true_1d = y_true.reshape(l * w, 1)
#    y_pred_1d = y_pred.reshape(l * w, 1)
#    y_int = np.intersect1d(y_true_1d, y_pred_1d)
    
    
#    tp = np.
#    fp =
#    fn =
#    p = tp / (tp + fp)
#    r = tp / (tp + fn)
#    f1 = 2 * ((p * r) / (p + r) )
    
    
graph = tf.Graph()
with graph.as_default():

    # Input data.
    # Load the training, validation and test data into constants that are
    # attached to the graph.
    tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
    tf_train_labels = tf.constant(train_labels[:train_subset, :])
    tf_valid_dataset = tf.constant(valid_dataset)
    tf_valid_labels = tf.constant(valid_labels)
    tf_test_dataset = tf.constant(test_dataset)
    
    # Variables.
    # These are the parameters that we are going to be training. The weight
    # matrix will be initialized using random valued following a (truncated)
    # normal distribution. The biases get initialized to zero.
    
    weights1 = tf.Variable(
        tf.truncated_normal([input_size,  layer_1_units]))
    biases1 = tf.Variable(tf.zeros([layer_1_units]))
    
    #Inputs are of length 128 (number hidden units of layer1), but outputs need to be of size 9 (number of labels))
    weights2 = tf.Variable(
        tf.truncated_normal([layer_1_units, layer_2_units]))
    biases2 = tf.Variable(tf.zeros([layer_2_units]))
    
    weights3 = tf.Variable(
        tf.truncated_normal([layer_2_units, labels_size]))
    biases3 = tf.Variable(tf.zeros([labels_size]))
  
   # Training computation.
    def layer(inputs, dropout=False):
        if dropout:
            inputs = tf.nn.dropout(inputs, dropout_keep_prob)
            
        logits1 = tf.matmul(inputs, weights1) + biases1
        layer1_out = tf.nn.relu(logits1)
        
        logits2 = tf.matmul(layer1_out, weights2) + biases2
        layer2_out = tf.nn.relu(logits1)
        
        logits_out = tf.matmul(layer2_out, weights3) + biases3
        return logits_out
    
    tf.nn.sigmoid(layer(tf_train_dataset)
    
    loss = tf.reduce_mean(
        #This uses sigmoid cross entropy, which allows for multiple labels
        tf.nn.sigmoid_cross_entropy_with_logits(layer(tf_train_dataset, True), tf_train_labels))
    

    # L2 regularization for the fully connected parameters.
    regularizers = (tf.nn.l2_loss(weights1) + tf.nn.l2_loss(biases1) +
                  tf.nn.l2_loss(weights2) + tf.nn.l2_loss(biases2))
    
    # Add the regularization term to the loss.
    #loss += 5e-7 * regularizers
    
    loss = loss * loss

    # Optimizer.
    # We are going to find the minimum of this loss using gradient descent.
    optimizer = tf.train.AdagradOptimizer(.05).minimize(loss)

    # Predictions for the training, validation, and test data.
    # These are not part of training, but merely here so that we can report
    # accuracy figures as we train.
    train_prediction = tf.nn.sigmoid(layer(tf_train_dataset))
    valid_prediction = tf.nn.sigmoid(layer(tf_valid_dataset))
    test_prediction = tf.nn.sigmoid(layer(tf_test_dataset))

In [39]:
num_steps = 100000

with tf.Session(graph=graph) as session:
    # This is a one-time operation which ensures the parameters get initialized as
    # we described in the graph: random weights for the matrix, zeros for the
    # biases. 
    tf.initialize_all_variables().run()
    print 'Initialized'
    for step in xrange(num_steps):
        # Run the computations. We tell .run() that we want to run the optimizer,
        # and get the loss value and the training predictions returned as numpy
        # arrays.
        _, l, predictions = session.run([optimizer, loss, train_prediction])
        if (step % 1000 == 0):
            #print predictions.shape
            print "-----"
            print 'Loss at step', step, '/', num_steps ,':', l
            #print "F1 Score:", (valid_labels)
            print 'Training accuracy: ',f1_score.mean_f1_score(
               predictions, train_labels[:train_subset, :])
            # Calling .eval() on valid_prediction is basically like calling run(), but
            # just to get that one numpy array. Note that it recomputes all its graph
            # dependencies.
            print 'Validation accuracy:', f1_score.mean_f1_score(
                 valid_prediction.eval(), valid_labels)
            
            test_results = test_prediction.eval()
            
#Currently seems to overfit/ converge very quickly
#Training 99.8%
#Validation 87.5 %

Initialized
-----
Loss at step 0 : 634730.0
Training accuracy:  0.423248228736
Validation accuracy: 0.467644119769
-----
Loss at step 1000 : 2554.14
Training accuracy:  0.697263937508
Validation accuracy: 0.673922355422
-----
Loss at step 2000 : 1482.4
Training accuracy:  0.72040682782
Validation accuracy: 0.686368797869
-----
Loss at step 3000 : 1035.13
Training accuracy:  0.733711930056
Validation accuracy: 0.690709154408
-----
Loss at step 4000 : 797.327
Training accuracy:  0.739506486414
Validation accuracy: 0.689246991571
-----
Loss at step 5000 : 635.38
Training accuracy:  0.745282371554
Validation accuracy: 0.690675535249
-----
Loss at step 6000 : 521.616
Training accuracy:  0.748350028656
Validation accuracy: 0.691467562666
-----
Loss at step 7000 : 439.227
Training accuracy:  0.747696542402
Validation accuracy: 0.688730910429
-----
Loss at step 8000 : 376.61
Training accuracy:  0.75473365094
Validation accuracy: 0.688078160402
-----
Loss at step 9000 : 327.339
Training accurac

In [42]:
import skflow
import random

random.seed(42) # to sample data the same way
clf = skflow.TensorFlowLinearClassifier(n_classes=9, batch_size=128, steps=500, learning_rate=0.05)
clf.fit(train_dataset, train_labels)
print f1_score.mean_f1_score(
                 clf.predict(valid_dataset), valid_labels)


ValueError: Shapes (?, 9) and (?, 9, 9) must have the same rank

In [238]:
#print test_bids

test_results = np.array(np.round(test_results), dtype=bool)
output = "business_id,labels\n"

for i in range(len(test_bids)):
    print i
    tags = np.argwhere(test_results[i])
    tags_str = ' '.join('%d'%F for F in tags[:] )
    output += test_bids[i]+ "," + tags_str + "\n"
    
print output
f = open('test_submission_net_max', 'w')
f.write(output)
f.close()

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
output

In [ ]:
test_dataset[1]